In [12]:
import os
import PyPDF2
import time
import pandas as pd
import docx
import re
import glob
import google.generativeai as genai
import os
genai.configure(
    api_key=os.environ['API_KEY'])
model_gemini = genai.GenerativeModel(
    model_name='gemini-pro')

# Function to apply gemini-3.5 model to a given text

In [13]:

def generate_text(work, desc, rubrics):
    rubrics = rubrics
    text = work
    instructor = """an experienced language instructor and assess the work of students. 
    I will provide the student's essay, Assignment description given to students, and the rubrics. 
    Your task is to assign a score and provide detailed comments for each rubric. 
    Highlight words, sentences, or phrases where the student made mistakes or did not meet the rubric criteria."""

    prompt_aspect = f"""Act as {instructor} 

    Description:
    {desc}

    Rubrics:
    {rubrics}

    Student Work: 
    {text}

    Instructions:
    1. Assign a score for each rubric criterion based on the student's essay.
    2. Provide detailed comments explaining the score for each criterion.
    3. Highlight specific mistakes or areas where the essay did not meet the rubric criteria.
    4. Offer constructive feedback for improvement where applicable."""


    try:
        completion = model_gemini.generate_content(
            prompt_aspect,
            generation_config={
                'temperature': 0.1,
                'max_output_tokens': 800
            }
        )
        score_gemini = completion.text
    except Exception as e:
        score_gemini = str(e) 
    result = score_gemini    
    # result = desc + rubrics+ text
    return result



# Functions to read files

In [14]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text



In [15]:

# Define function to process all files in a given folder
def process_file(filename):
    print(filename)
    try:
        if filename.endswith('.docx'):
                text = read_word_file(filename)
        elif filename.endswith('.pdf'):
                doc = fitz.open(filename)
                text = ""
                for page in doc:
                        text+=page.get_text()
                        print(text)
        else:
                text = ""
    except Exception as e:
        text = str(e)
    return text    


## Main Folder of Assignment

In [16]:
folder_path = 'OLevels/'

In [17]:
import pandas as pd

df = pd.read_csv(os.path.join(folder_path, "Results_Gemini/Results analysis_Olevels.csv"))
# df.dropna(subset=['human_Grade'], inplace=True)
print(len(df))

# print(df.head(5))


21


In [18]:
df = df.loc[df["Essay type"] == "Report Writing"]
df.head()

,Essay ID,Essay type,human1_Score,human2_Score,GPT_Score
0,e001,Report Writing,23,NaN,23
2,e003,Report Writing,23,NaN,21
3,e004,Report Writing,25,NaN,23
5,e006,Report Writing,26,NaN,23
7,e008,Report Writing,28,NaN,23


In [19]:
desc = read_word_file(os.path.join(folder_path, 'Description and Rubrics/Report writing_description.docx'))
rubrics = read_word_file(os.path.join(folder_path, 'Description and Rubrics/Report writing_rubrics.docx'))
# Create a list of unique rubric names
unique_rubrics = ["Score", "Point", "Rubrics"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["Comments", "Mistakes", "Quality of Language" "Appropriateness and Content"]
results = {rubric: [] for rubric in keywords}
feedback= []

In [20]:
rubrics

'It is recommended that you do not edit rubrics in spreadsheet format\t\t\t\t\t\t\t\t\nv1.0-s\t\t\t\t\t\t\t\t\nTask Fulfilment 15 MARKS\t\t\t\t\t\t\t\t\nCandidates will be awarded up to 15 marks for following the task instructions\t\t\t\t\t\t\t\t\n\t15\t12\t9\t6\t3\t\t\t\n\tBAND 5\tBAND 4\tBAND 3\tBAND 2\tBAND 1\t\t\t\n\t"Very good understanding of purpose.\n• Clear awareness of the specified situation and audience.\n• Text type entirely appropriate.\n• All required points developed in detail, fully amplified and well\norganised.\n• Given information well used to justify personal opinion and\ninterpretation.\n• Tone and register entirely appropriate."\t"Good understanding of purpose.\n• An awareness of the specified situation and audience.\n• Text type appropriate.\n• All required points addressed but not always developed in detail.\n• Given information organised to support personal opinion.\n• Tone and register appropriate"\t"• Some understanding of purpose; may lack some focus.\n• So

In [21]:
desc

'1.Explore the sample response on a past paper question for Report writing to revise the format and structure of reports.\n\n\n2.Compose your response on the given prompt.\n\nYou are advised to write between 200 and 300 words.\n\nYou will be awarded up to 15 marks for following the task instructions.\nYou will be awarded up to 15 marks for the language you use.\n\nThe majority of your schoolmates are not interested in sports. You have carried out a survey on the reason for their lack of interest. Based on your findings, write a report to the Principal regarding the matter.\n\nYou must include the following in your report:\n\nReason for the lack of interest in sports\nSome information on school facilities\nsuggestions to overcome the problems\nSample from Past paper:\nTask\nMany new students join your school every year. Your principal wants to make starting at the school as easy as possible for the new students and she asks you as a member of Student Council, to write a report for her a

# Call the process_files function

In [22]:
jump=1
path = "E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing/"
for i in range(0,  len(df), jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():
        # filename = os.path.join("OLevels/Collected essays/Informal/", row["Essay ID"])
        filename = next(glob.iglob(os.path.join(path, row["Essay ID"]+'.*')), None)
        # Get the text to assess
        if filename:
            print(filename)
            text_to_assess = process_file(filename)
        else:
            text_to_assess = "Not Submitted the work"
        print(text_to_assess)
            # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, desc=desc, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result, "\n\n\n")
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")   
        time.sleep(20) 
    for rubric in unique_rubrics:
        data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword] = results[keyword]   
    data['OverallResponse'] = feedback
    
    # Save the data with the rubric columns to a new CSV file
    data.to_csv(os.path.join(folder_path, "Results_Gemini/Results analysis_Olevels_Gemini.csv"), index=False, header=False, mode='a')
    # data.to_csv(os.path.join(folder_path, "Results_Gemini/___Report_Essay_Test.csv"), index=False, mode='a')

    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]

E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e001.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e001.docx
Report

To: The principal, of IBA Public School Sukkur
From: Student of Cambridge Section (O-Level 1) 
Date: June/4th/ 2021 

Subject: Students interest in sports

Introduction
 
Many of my schoolmates are not interested in sports. I had carried out a survey on the reasons for their lack of interest. The reasons that causes students not to be interested in sports comes from themselves, parents and facilities. Firstly, the attitude of students for sports was wasting their time. They feel that time is money, so they use their time to study. They think that studying is more important than sports and study ensures their future. This causes them to have a lack of time for sports.

Parent’s point of view

Parents are also the reason why students lack interest in sports. Parents emphasize their child’s aca

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e003.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e003.docx
Report Writing

To: Principal
From:Moiz Ahmed, member of Student Council
Date: 13th June, 2021

Subject: Convincing students to take part in sports                           This report is going to introduce why the majority of students in our school are not interested in sports.

There are a few reasons why students are not interested in sports. Students spend most of their time in studies and spend their most of the time in sitting. Students also spend their time in excessive use of internet. Students also spend most of their time in watching televison of playing games on their mobiles or desktops. Due to all these activities they skip their co-curricular or physical activities.Most of the parents emphasize on their child’s academic performance rather than their co-curriculum activities. Parents t

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e004.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e004.docx
To: The Principal
From: Fiza Fatima, student of o-level-1 
Date: 2nd June 2021

Subject: Bringing sports into interest

This report would be elaborating the reasons for the lack of interest of students in sports. Furthermore, we shall look over numerous possible ways to expand student’s interest in such activities involving physical excretion and developing skills. 

Sports are essential to be played by every student, as they consist of teamwork, leadership, social skills, brainpower and passion which could lead students to success and a better future. 

Since most of the students in our school aren’t interested in sports thereupon, we have surveyed the reasons for their lack of interest in sports. Most of the students proclaimed that because of the unbearable high temperature they have become unabl

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e006.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e006.docx
To: The PrincipalIBA Public 
From: Salahuddin, member of Student Council
Date: 6th June 2021
 
Subject: Starting off the school as easy as possible for new students
 
Introduction
The following report is about the lack of interest among the students of IBA PSS and suggestions to overcome it.
 
Reasons for having lack of interest
I have recentlsurveyedng the students of IBA PSS. The results of the survey show that among 1,000 students from Class one to eight, only 200 would choose sports as their preferred leisure activity during their spare time. Students assume playing sports involves getting hot, dirty and sweaty as well as being occasionally sunburned. Also, the students complained about their lack of time. Many of them commented that this was caused by the longer school hours and also because of

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e008.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Report Writing\e008.docx
                                        Report Writing

To: The Principle
From: Iman Nawaz, Head prefect of the school.
Date: 4th June, 2021

Subject: A survey among my schoolmates to find out the reasons for their lack of interest in sports.

Introduction: The aim of creating this report is to discover the reasons behind the lack of interest in sports among my schoolmates and how to overcome those difficulties.

Reasons for their lack of interest: 
            It is very gloomy to see that numerous students are not involved in sports activities though it's significant for our physical and mental health both. After this survey, we got to know that the majority of students think that playing sports is a waste of time and they believe it's better to give priority to their studies. Moreover, few studen

C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_11252\3089589956.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 